# Imports

In [1]:
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Import .txt file and convert it to a DataFrame object
df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    900223 non-null  object 
 1   time    900223 non-null  object 
 2   page    900222 non-null  object 
 3   id      900223 non-null  int64  
 4   cohort  847330 non-null  float64
 5   ip      900223 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 41.2+ MB


In [12]:
df.date = pd.to_datetime(df.date)
df = df.set_index('date')

AttributeError: 'DataFrame' object has no attribute 'date'

# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [40]:
df = df[df.page != '/']

In [95]:
# what are the lessons, what are the cohorts, and what cohorts and normally associated with what programs
# group by cohort, rank by lesson 
df.cohort.value_counts()

28.0     77691
33.0     39077
29.0     36298
62.0     35413
53.0     34864
24.0     33535
56.0     32222
57.0     32117
51.0     31421
22.0     29098
59.0     29035
58.0     28490
32.0     28401
23.0     26893
26.0     26825
52.0     26738
34.0     24571
25.0     24454
31.0     24185
132.0    22752
55.0     19870
27.0     19762
61.0     16941
134.0    16132
135.0    15698
133.0    13600
14.0      9462
1.0       8264
137.0     7978
21.0      6814
138.0     6804
17.0      4705
13.0      2626
18.0      2009
139.0     1541
8.0       1502
19.0      1086
16.0       718
15.0       658
7.0        547
12.0       294
11.0       234
2.0         76
6.0         68
9.0          4
4.0          3
Name: cohort, dtype: int64

In [48]:
df.cohort.value_counts()

28.0     77691
33.0     39077
29.0     36298
62.0     35413
53.0     34864
24.0     33535
56.0     32222
57.0     32117
51.0     31421
22.0     29098
59.0     29035
58.0     28490
32.0     28401
23.0     26893
26.0     26825
52.0     26738
34.0     24571
25.0     24454
31.0     24185
132.0    22752
55.0     19870
27.0     19762
61.0     16941
134.0    16132
135.0    15698
133.0    13600
14.0      9462
1.0       8264
137.0     7978
21.0      6814
138.0     6804
17.0      4705
13.0      2626
18.0      2009
139.0     1541
8.0       1502
19.0      1086
16.0       718
15.0       658
7.0        547
12.0       294
11.0       234
2.0         76
6.0         68
9.0          4
4.0          3
Name: cohort, dtype: int64

In [112]:
for cohort in df.cohort.unique():
    print(cohort)
    print(df[df.cohort == cohort].page.value_counts().sort_values(ascending=False).head(1))
    print('---')


8.0
java-iii    57
Name: page, dtype: int64
---
22.0
spring    744
Name: page, dtype: int64
---
21.0
spring    364
Name: page, dtype: int64
---
1.0
javascript-i    294
Name: page, dtype: int64
---
16.0
spring    65
Name: page, dtype: int64
---
18.0
javascript-i    69
Name: page, dtype: int64
---
19.0
content/laravel/intro    38
Name: page, dtype: int64
---
nan
Series([], Name: page, dtype: int64)
---
13.0
index.html    84
Name: page, dtype: int64
---
14.0
index.html    877
Name: page, dtype: int64
---
15.0
java-i    19
Name: page, dtype: int64
---
7.0
content/html-css    29
Name: page, dtype: int64
---
4.0
mkdocs/search_index.json    1
Name: page, dtype: int64
---
12.0
content/html-css    19
Name: page, dtype: int64
---
17.0
javascript-i    128
Name: page, dtype: int64
---
23.0
mkdocs/search_index.json    721
Name: page, dtype: int64
---
2.0
content/php_ii/command-line    6
Name: page, dtype: int64
---
9.0
content/html-css    2
Name: page, dtype: int64
---
11.0
html-css    12
Name: pag

In [109]:
df[df.cohort == 28].page.value_counts().sort_values(ascending=False).head(1)

javascript-i    1817
Name: page, dtype: int64

In [107]:
df.groupby(['cohort']).value_counts().sort_values(ascending=False)

cohort  time      page                                                                       id   ip            
53.0    17:41:11  slides/arrays                                                              570  172.124.67.93     12
        17:42:00  slides/arrays                                                              570  172.124.67.93     10
        17:41:10  slides/arrays                                                              570  172.124.67.93     10
        17:41:09  slides/arrays                                                              570  172.124.67.93      9
        17:42:01  slides/arrays                                                              570  172.124.67.93      8
                                                                                                                    ..
28.0    18:13:43  javascript-i/loops                                                         146  97.105.19.58       1
                  mysql/tables                        

In [71]:
df2 = df.copy()

# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?


# 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?


# 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?


# 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?


# 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?


# 7. Which lessons are least accessed? 
